In [1]:
import pandas as pd
import requests
import gc

# Variável global para armazenar o DataFrame
df_comex = None

In [2]:
def obter_dados_comex(ano: str, mes: str, tipo_operacao: str) -> str:
    """
    Baixa e carrega os dados de comércio exterior (EXP ou IMP) para um
    determinado ano e mês.

    Args:
        ano (str): O ano dos dados, por exemplo, '2024'.
        mes (str): O mês dos dados, por exemplo, 'janeiro'.
        tipo_operacao (str): 'EXP' ou 'IMP'.

    Returns:
        str: Uma mensagem de sucesso ou erro.
    """
    global df_comex
    print("\n================ INÍCIO obter_dados_comex ================")
    print(f"Parâmetros recebidos: ano={ano}, mes={mes}, tipo_operacao={tipo_operacao}")
    tipo_operacao = tipo_operacao.upper()
    meses = {
        "janeiro": 1,
        "fevereiro": 2,
        "março": 3,
        "abril": 4,
        "maio": 5,
        "junho": 6,
        "julho": 7,
        "agosto": 8,
        "setembro": 9,
        "outubro": 10,
        "novembro": 11,
        "dezembro": 12,
    }
    mes_num = meses.get(mes.lower())

    if not mes_num:
        print("[ERRO] Mês inválido!")
        print("================ FIM obter_dados_comex ================\n")
        return "Mês inválido. Por favor, use o nome completo do mês em português."
    if tipo_operacao not in ["EXP", "IMP"]:
        print("[ERRO] Tipo de operação inválido!")
        print("================ FIM obter_dados_comex ================\n")
        return "Tipo de operação inválido. Use 'EXP' ou 'IMP'."

    url = f"https://balanca.economia.gov.br/balanca/bd/comexstat-bd/mun/{tipo_operacao}_{ano}_MUN.csv"
    try:
        print(f"[DOWNLOAD] Baixando dados anuais de {tipo_operacao} para {ano}...")
        df_anual = pd.read_csv(url, sep=";", encoding="iso-8859-1", on_bad_lines="skip")

        print(
            f"[OK] Dados anuais de {tipo_operacao} carregados. Total de linhas: {len(df_anual)}"
        )
        df_comex = df_anual[df_anual["CO_MES"] == mes_num].copy()
        if df_comex.empty:
            print(f"[AVISO] Nenhum dado encontrado para o mês de {mes}/{ano}!")
            print("================ FIM obter_dados_comex ================\n")
            return f"Nenhum dado de {tipo_operacao} encontrado para o mês de {mes}/{ano}. Verifique se a combinação de mês e ano possui dados."
        print(
            f"[OK] Dados filtrados para o mês de {mes}/{ano}. Total de linhas: {len(df_comex)}"
        )
        print("================ FIM obter_dados_comex ================\n")
        return f"Dados de {tipo_operacao} para {mes}/{ano} carregados com sucesso. Agora você pode fazer perguntas sobre eles."
    except requests.exceptions.HTTPError as e:
        print(f"[ERRO HTTP] {e}")
        if e.response is not None and e.response.status_code == 404:
            print("================ FIM obter_dados_comex ================\n")
            return f"Erro: O arquivo de dados para {tipo_operacao} em {ano} não foi encontrado. A URL tentada foi: {url}. Por favor, verifique se os dados para este período estão disponíveis."
        print("================ FIM obter_dados_comex ================\n")
        return f"Erro HTTP ao baixar os dados: {e}. URL tentada: {url}"
    except Exception as e:
        print(f"[ERRO EXCEÇÃO] {e}")
        print("================ FIM obter_dados_comex ================\n")
        return f"Ocorreu um erro ao processar os dados: {e}"

In [3]:
import pandas as pd
import requests
import io

# Defina as variáveis que você usaria na função
ano = '2023'
tipo_operacao = 'EXP'

# Crie a URL
url = f"https://balanca.economia.gov.br/balanca/bd/comexstat-bd/mun/{tipo_operacao}_{ano}_MUN.csv"

# Faça a requisição GET para a URL
try:
    response = requests.get(url)
    response.raise_for_status()  # Levanta um erro se a requisição não for bem-sucedida

    # Use io.StringIO para ler o conteúdo da resposta como um arquivo em memória
    csv_content = io.StringIO(response.text)

    # Tente ler o CSV com pandas, usando os mesmos parâmetros do seu código
    df_teste = pd.read_csv(csv_content, sep=';', encoding='iso-8859-1', on_bad_lines='skip', engine='python')
    
    print("Download e leitura do arquivo CSV da URL bem-sucedidos!")
    print(f"Número de linhas carregadas: {len(df_teste)}")
    print("Mostrando as primeiras 5 linhas:")
    print(df_teste.head())

except requests.exceptions.HTTPError as e:
    print(f"Erro HTTP: {e}")
    print("A URL pode estar incorreta ou o arquivo não está disponível.")
except Exception as e:
    print(f"Ocorreu um erro ao processar os dados: {e}")
    print("A URL está correta, mas a formatação do arquivo CSV está causando problemas na leitura.")

Download e leitura do arquivo CSV da URL bem-sucedidos!
Número de linhas carregadas: 1121352
Mostrando as primeiras 5 linhas:
   CO_ANO  CO_MES   SH4  CO_PAIS SG_UF_MUN   CO_MUN  KG_LIQUIDO  VL_FOB
0    2023      10  2209      127        BA  2919553          36      53
1    2023      11  4009      493        SP  3456701         473   14008
2    2023      11  8536      365        SP  3438709           2     181
3    2023       8  8512       63        RJ  3304201       15627  239105
4    2023       6  1902      580        PR  4118204         287     368


In [4]:
def resumo_dados_comex(consulta: str) -> str:
    """
    Executa uma consulta específica nos dados de comércio exterior carregados.

    Args:
        consulta (str): Uma pergunta em linguagem natural sobre os dados.

    Returns:
        str: A resposta à consulta ou uma mensagem de erro.
    """
    print("\n================ INÍCIO resumo_dados_comex ================")
    global df_comex
    if df_comex is None:
        print("[ERRO] Nenhum dado carregado!")
        print("================ FIM resumo_dados_comex ================\n")
        return "Nenhum dado de comércio exterior foi carregado. Por favor, use a ferramenta 'obter_dados_comex' primeiro."
    consulta_lower = consulta.lower()
    print(f"Consulta recebida: {consulta}")
    if "média do peso líquido" in consulta_lower:
        if "KG_LIQUIDO" in df_comex.columns:
            media = df_comex["KG_LIQUIDO"].mean()
            print(f"[OK] Média do peso líquido: {media:.2f} kg")
            print("================ FIM resumo_dados_comex ================\n")
            return f"A média do peso líquido dos dados carregados é de {media:.2f} kg."
        else:
            print("[ERRO] Coluna 'KG_LIQUIDO' não encontrada!")
            print("================ FIM resumo_dados_comex ================\n")
            return "A coluna 'KG_LIQUIDO' não foi encontrada nos dados."
    elif "principais estados" in consulta_lower:
        if "SG_UF_NCM" in df_comex.columns:
            top_estados = df_comex["SG_UF_NCM"].value_counts().head(5)
            print(f"[OK] Top 5 estados:\n{top_estados}")
            print("================ FIM resumo_dados_comex ================\n")
            return f"Os 5 principais estados por número de operações são:\n{top_estados.to_string()}"
        else:
            print("[ERRO] Coluna 'SG_UF_NCM' não encontrada!")
            print("================ FIM resumo_dados_comex ================\n")
            return "A coluna 'SG_UF_NCM' não foi encontrada nos dados."
    else:
        print("[ERRO] Consulta não reconhecida!")
        print("================ FIM resumo_dados_comex ================\n")
        return "Não foi possível processar a sua consulta. Tente perguntas sobre 'média do peso líquido' ou 'principais estados'."

In [5]:
def limpar_dados_comex() -> str:
    """
    Libera os dados de comércio exterior da memória para economizar recursos computacionais.
    """
    print("\n================ INÍCIO limpar_dados_comex ================")
    global df_comex
    if df_comex is None:
        print("[AVISO] Não há dados para limpar!")
        print("================ FIM limpar_dados_comex ================\n")
        return "Não há dados para limpar na memória."
    df_comex = None
    gc.collect()
    print("[OK] Dados de comércio exterior foram limpos da memória.")
    print("================ FIM limpar_dados_comex ================\n")
    return "Os dados foram removidos da memória com sucesso."

In [9]:
obter_dados_comex(ano='2023', mes='janeiro', tipo_operacao='EXP')


================ INÍCIO obter_dados_comex ================
Parâmetros recebidos: ano=2023, mes=janeiro, tipo_operacao=EXP
[DOWNLOAD] Baixando dados anuais de EXP para 2023...
[OK] Dados anuais de EXP carregados. Total de linhas: 1121352
[OK] Dados filtrados para o mês de janeiro/2023. Total de linhas: 84625
================ FIM obter_dados_comex ================



'Dados de EXP para janeiro/2023 carregados com sucesso. Agora você pode fazer perguntas sobre eles.'

In [10]:
resumo_dados_comex(consulta='Qual a média do peso líquido?')


================ INÍCIO resumo_dados_comex ================
Consulta recebida: Qual a média do peso líquido?
[OK] Média do peso líquido: 618844.48 kg
================ FIM resumo_dados_comex ================



'A média do peso líquido dos dados carregados é de 618844.48 kg.'

In [11]:
limpar_dados_comex()


================ INÍCIO limpar_dados_comex ================
[OK] Dados de comércio exterior foram limpos da memória.
================ FIM limpar_dados_comex ================



'Os dados foram removidos da memória com sucesso.'